In [72]:
# #Set the enviroment variables
# import os
# os.environ['KAGGLE_USERNAME'] = "dlromanoff"
# os.environ['KAGGLE_KEY'] = "a63612e0bc2ceaee594bcec8d8f498b6"

In [162]:
# Imports needed or not
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from scipy import stats
import statsmodels.api as sm
import statsmodels.api as sm
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D

In [64]:
# Trying to see all the columns of the dataframe
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [199]:
# Read the csv's
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')
train_labels = pd.read_csv('train_labels.csv')
train_features.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [200]:
# Get some idea of the numbers I am working with
train_features.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [201]:
# See what's going in the labels dataframe
train_labels.describe(include='object')

,status_group
count,59400
unique,3
top,functional
freq,32259


Labels dataframe includes the status_group along with the id. The status group is the dependent variable here. The id needs to be used to track the status group.

In [202]:
# Checking for values
train_labels.status_group.value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

**This indicates what our expectations should be for our outcomes. Function here is ~53.31%. Non-Function is ~38.42%. And, functional needs repair is ~7.27%**

In [203]:
# Baseline Prediction
majority_class = train_labels['status_group'].mode()[0]
y_pred = np.full(shape=train_labels['status_group'].shape, fill_value=majority_class)
print(train_labels.status_group.shape)
y_pred.shape

(59400,)


(59400,)

In [204]:
accuracy_score(train_labels['status_group'], y_pred)

0.543080808080808

In [205]:
# Making submission DataFrame
submission = pd.DataFrame(y_pred, columns=['status_group_included'])
submission = pd.concat([train_labels, submission], axis=1)
submission.head()

,id,status_group,status_group_included
0,69572,functional,functional
1,8776,functional,functional
2,34310,functional,functional
3,67743,non functional,functional
4,19728,functional,functional


In [206]:
submission = submission.drop(columns='status_group')
print(submission.shape)
submission.head()

(59400, 2)


,id,status_group_included
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,functional
4,19728,functional


In [207]:
submission.to_csv(path_or_buf="/Users/danielleromanoff/Documents/Lambda/DS-Unit-2-Project-Week/Submission.csv", sep=',', index=False)

In [208]:
# Making all objects into numbers
def encode_cat(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df

In [209]:
y = test_features.drop(columns='id')
X_train, X_test, y_train, y_test = train_test_split(
        train_features, y, train_size=0.75, test_size=0.25, random_state=42, shuffle=True)
    
X_train.shape, X_test.shape, y_train.shape, y_test.shape

ValueError: Found input variables with inconsistent numbers of samples: [59400, 14358]

In [157]:
# Merging dataframes by id to make it easier to use
train_new = pd.merge(train_labels, train_features, on='id')
print(train_new.shape)
train_new.head()

(59400, 41)


,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [158]:
# Using only numeric columns for now
train_numeric = train_new.select_dtypes(include=['number'])
print(train_numeric.shape)
train_numeric.head()

(59400, 10)


,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
0,69572,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999
1,8776,0.0,1399,34.698766,-2.147466,0,20,2,280,2010
2,34310,25.0,686,37.460664,-3.821329,0,21,4,250,2009
3,67743,0.0,263,38.486161,-11.155298,0,90,63,58,1986
4,19728,0.0,0,31.130847,-1.825359,0,18,1,0,0


In [159]:
# Checking for null values
train_numeric.isnull().sum()

id                   0
amount_tsh           0
gps_height           0
longitude            0
latitude             0
num_private          0
region_code          0
district_code        0
population           0
construction_year    0
dtype: int64

In [163]:
X = train_numeric
y = train_labels.status_group
model = LogisticRegression(solver='lbfgs')
model.fit(X, y)
y_pred = model.predict(X)
print('Classification Report')
print()
print(classification_report(y, y_pred))
print('The accuracy is: ', accuracy_score(y, y_pred))

/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Classification Report



/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                         precision    recall  f1-score   support

             functional       0.56      0.93      0.70     32259
functional needs repair       0.00      0.00      0.00      4317
         non functional       0.49      0.13      0.20     22824

              micro avg       0.55      0.55      0.55     59400
              macro avg       0.35      0.35      0.30     59400
           weighted avg       0.49      0.55      0.45     59400

The accuracy is:  0.550993265993266


/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [171]:
prediction = y_pred
prediction = pd.DataFrame(prediction)
print(prediction.shape)
prediction.head()

(59400, 1)


,0
0,functional
1,functional
2,functional
3,non functional
4,functional


In [172]:
# Combining the label from the numeric dataframe and the prediction
prediction = pd.concat([train_numeric.id, prediction], axis=1)
prediction.columns.values[1]= 'status_group'
prediction.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [175]:
# Finding null values after the merge.
train_new.isnull().sum()

id                           0
status_group                 0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_qu

In [149]:
# For now just dropping NaN's and checking to make sure there are none left
# train_clean = train_new.dropna(axis=1)
# train_clean.isnull().sum()

In [150]:
# Making a list of the columns in the new df
# print(list(train_clean))

In [94]:
# Assigning X and y values for Logistic Regression
# X = train_clean.drop(['status_group'], axis=1)
# y = train_clean['status_group']

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, random_state=42)
print('X_train shape is ',X_train.shape)
print('y_train shape is ', y_train.shape, '\n')
print('X_test shape is ',X_test.shape)
print('y_test shape is ', y_test.shape)

X_train shape is  (44550, 33)
y_train shape is  (44550,) 

X_test shape is  (14850, 33)
y_test shape is  (14850,)


In [113]:
# Encoding X_train and X_test
X_train = encode_cat(X_train)
X_test = encode_cat(X_test)
print('X_train shape is ',X_train.shape)
print('X_test shape is ',X_test.shape)

X_train shape is  (44550, 33)
X_test shape is  (14850, 33)


In [116]:
X_train.head()

,id,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,recorded_by,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
24947,33935,20.0,41,330,38.123839,-6.087137e+00,1513,0,8,14,6,1,3,1172,80,0,1972,6,10,5,7,4,4,5,4,4,2,2,3,0,0,1,1
22630,49654,0.0,291,0,0.000000,-2.000000e-08,30655,0,4,13,17,1,54,1660,0,0,0,8,5,1,11,4,6,6,6,2,1,1,7,5,0,4,3
13789,39287,0.0,135,0,33.312321,-2.814100e+00,1698,0,4,13,19,4,45,2070,0,0,0,4,2,1,7,4,0,2,6,2,1,1,3,0,0,4,3
15697,60510,0.0,264,1542,34.783049,-4.842093e+00,37195,0,0,18,13,4,110,1428,23,0,2010,3,1,0,10,4,0,2,6,2,3,3,5,3,1,1,1
22613,24259,0.0,348,523,34.660944,-1.070733e+01,17901,0,1,16,10,3,61,1139,1,0,2013,3,1,0,7,4,6,6,6,2,0,0,8,6,0,1,1


In [117]:
# Fitting the model and getting its accuracy score
model= LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.635959595959596

In [125]:
# Using pipeline with the training data
pipeline = make_pipeline(ce.OneHotEncoder(use_cat_names=True),
                         StandardScaler(), LogisticRegression(solver ='lbfgs',n_jobs=-1, multi_class = 'auto',C=2))
pipeline.fit(X_train, y_train)

/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


Pipeline(memory=None,
     steps=[('onehotencoder', OneHotEncoder(cols=[], drop_invariant=False, handle_unknown='impute',
       impute_missing=True, return_df=True, use_cat_names=True, verbose=0)), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logisticregression', LogisticRegression(C=2, cla...enalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False))])

In [128]:
# y prediction using the pipeline
y_pred = pipeline.predict(X_train)

/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [131]:
# Putting results in a dataframe
prediction = pd.DataFrame(y_pred, X_train['id'])
prediction.columns = ['status_group']
print(prediction.shape)
prediction.head()

(44550, 1)


,status_group
id,
33935,functional
49654,functional
39287,non functional
60510,functional
24259,non functional


In [174]:
prediction.to_csv(path_or_buf="/Users/danielleromanoff/Documents/Lambda/DS-Unit-2-Project-Week/Prediction_1.csv", sep=',', index=False)

In [143]:
# Making the 'status_group' a column
fix_df= pd.DataFrame(prediction)
submit = fix_df.index
submission_1 = pd.DataFrame(fix_df['status_group'],submit)
submission_1.reset_index(inplace = True)
submission_1.head()

,id,status_group
0,33935,functional
1,49654,functional
2,39287,non functional
3,60510,functional
4,24259,non functional


In [144]:
# Dropping all status_group values that are not equal to 'functional'
submission_1 = submission_1[submission_1["status_group"] == 'functional']
print(submission_1.shape)
submission_1.head()

(31302, 2)


,id,status_group
0,33935,functional
1,49654,functional
3,60510,functional
5,52400,functional
7,3435,functional


In [146]:
# newsub = pd.DataFrame(submission_1)
# print(newsub.shape)
# sub_2 = newsub.index
# submission_1 = pd.DataFrame(newsub['status_group'], sub_2)
# print(submission_1.head())
# subm.reset_index(inplace = True)

In [61]:
submission_1.to_csv(path_or_buf="/Users/danielleromanoff/Documents/Lambda/DS-Unit-2-Project-Week/Submission_1.csv", index=False)

In [ ]:
# Assigning X and y values for Logistic Regression
X = train_new.drop(['status_group'], axis=1)
y = train_new['status_group']